In [1]:
import qctoolkit as qtk
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output
import re
from datetime import datetime
import qmnet_model as qnm
import qmnet_tools as qnt
import qmnet_train as qntrain
import qmnet_layer as qnl
import tensorflow as tf

%matplotlib inline

clear_output()

In [2]:
from horton import GOBasisFamily
#basis = GOBasisFamily('basis', filename='basis/sto2g/H_Ne_uncontracted.nwchem')
#basis = '3-21g'
#basis = 'sto-3g'
#basis = GOBasisFamily('H_He_basis', filename='basis/sto6g/H_Ne.nwchem')
basis = GOBasisFamily('H_He_basis', filename='basis/sto6g/H_Be-s.nwchem')

resolution = 0.005
batch_size = 30000

In [3]:
molstr_list = [
    'HH',
]

In [4]:
inp_grp = []
#inp_dump = []
for molstr in molstr_list:
    if '+' in molstr:
        charge = 1
    elif '-' in molstr:
        charge = -1
    else:
        charge = 0
    alist = re.findall('[A-Z][a-z]*', molstr)
    Zs = [qtk.n2Z(a) for a in alist]
    inp_list = []
    inp_grp.append(inp_list)
    #inp_dump.append(inp_list)
    res = resolution
    for R in np.arange(0.3, 4.0 + res, res):
        mol = qtk.Molecule()
        ZR = [[Zs[0], 0,0,0], [Zs[1], R, 0, 0]]
        mol.build(ZR)
        mol.charge = charge
        mol.name = mol.name + '_%s' % str(R)
        inp = qtk.QMInp(mol, program='horton', basis_set=basis, scf_step=30)
        try:
            inp.run()
            D, U = np.linalg.eigh(inp.olp)
            inp.X = U / np.sqrt(D)
            
            # to save inps into pickle file
            #inp.delete_ht_types()
            #inp.delete_matrices()
            
            inp_list.append(inp)
        except:
            pass
    print "%s finished" % mol.name
print [len(inp) for inp in inp_grp]

H2_4.0 finished
[741]


In [13]:
reload(qnt)
reload(qnm)
reload(qnl)
#qm_model0 = qnm.Fock_test
inp_train, inp_test, inp_valid = qnt.train_test_valid(inp_grp, 0, [0])
I_train, Er_train, occ_train, nn_train, C_train, y_train = qnt.tfIO_batch(inp_train, batch_size)

In [153]:
reload(qnl)
tf.reset_default_graph()
g = tf.Graph()
with g.as_default():

    # setup model variables
    y = tf.placeholder(tf.float64, name='y')
#     with tf.variable_scope("m1"):
#         Fock, C_prd, param = qnm.Fock_test(g, inp_valid[0].olp.shape[0])
    #with tf.variable_scope("m2"):
    test, C_prd, param = qnm.Fock_error_test(g, inp_valid[0].olp.shape[0])
    I, Er, occ, nn, keep_prob = param
    
    ind = 2
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        train_dict = {I:I_train[0][:ind], Er:Er_train[0][:ind], occ:occ_train[0][:ind], nn:nn_train[0][:ind], y:y_train[0][:ind]}
        #Focks, C_prds = sess.run([Fock, C_prd], feed_dict=train_dict)
        C_prds, tests = sess.run([C_prd, test], feed_dict=train_dict)
        #Focks, C_prds = sess.run([Fock, C_prd], feed_dict=train_dict)
print tests

0.00205396439425


In [157]:
def err_i(i):
    inp = inp_train[i]
    C = C_prds[i]
    C_occ = C * inp.occ
    F = inp_train[i].Fock_matrix(C)
    S = inp.olp
    Fc_occ = F.dot(C_occ)
    Sc_occ = S.dot(C_occ)
    FS = np.diag(Fc_occ.T.dot(Sc_occ))
    SS = np.diag(Sc_occ.T.dot(Sc_occ))
    SS_copy = SS.copy()
    SS_copy[SS == 0.] = 1.
    e_list = FS/SS_copy
    eSc = e_list * Sc_occ
    res = Fc_occ - eSc
    err = np.sum(res * res)
    return err
err_i(0) + err_i(1)
#     print eSc
#     print err
#     print res

0.0020539643942457928

In [121]:
Fc_occ = F.dot(C_occ)
Sc_occ = S.dot(C_occ)
FS = np.diag(Fc_occ.T.dot(Sc_occ))
SS = np.diag(Sc_occ.T.dot(Sc_occ))
SS_copy = SS.copy()
SS_copy[SS == 0.] = 1.
e_list = FS/SS_copy
res = Fc_occ - e_list * Sc_occ
np.sum(res * res)

0.00066989944271930496

In [122]:
tests[:,:, :1]

array([[[ 0.19348906],
        [ 0.20006051],
        [ 0.18652208],
        [ 0.18720848]],

       [[ 0.08207683],
        [ 0.05757425],
        [-0.0116225 ],
        [-0.11809828]]])